In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)

# Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import os
import get_data
import pickle

In [2]:
parent_folder = os.path.normpath(os.getcwd() + os.sep + os.pardir)
root_path = (os.path.join(parent_folder,'raw_data'))
root_path

'/home/jxsal/code/jxsaliba/HeatPumpEnergyPredict/raw_data'

In [3]:
os.chdir(root_path)

In [4]:
energy_tele = pickle.load(open('energy_tele.pkl','rb'))

In [5]:
sensor_tele = pickle.load(open('sensor_tele.pkl','rb'))

In [6]:
hub_tele = pickle.load(open('hub_tele.pkl','rb'))

In [7]:
os.listdir()

['Untitled.ipynb',
 'sensor_telemetry.csv',
 'sensor_tele.pkl',
 'installation.csv',
 'hub_telemetry.csv',
 'hub.csv',
 'set_properties_running_Sep2022_Jan2023.pkl',
 '.ipynb_checkpoints',
 'epc.csv',
 'energy_sensor_telemetry.csv',
 'hub_tele.pkl',
 'property_old.csv',
 'daily_df.pkl',
 'energy_tele.pkl',
 'property.csv',
 'pmd_weather_humidity_energy_use.csv',
 'sensor.csv',
 'property_metrics_daily.csv']

In [8]:
common_set_list = pickle.load(open('set_properties_running_Sep2022_Jan2023.pkl','rb'))

In [9]:
energy_tele = energy_tele[energy_tele['property_id'].isin(common_set_list)]

In [10]:
uniq_units = energy_tele['unit'].unique()
print(uniq_units)

['WH100']


In [11]:
hub_tele = hub_tele[hub_tele['property_id'].isin(common_set_list)]

In [12]:
sensor_tele = sensor_tele[sensor_tele['property_id'].isin(common_set_list)]

In [13]:
merged_df = hub_tele.merge(sensor_tele, on = ['property_id', 'occurred'], how = 'inner')
merged_df = merged_df.merge(energy_tele, on=['property_id','occurred'], how = 'inner')


In [14]:
merged_df.columns

Index(['property_id', 'type', 'occurred', 'is_hot_water_on',
       'is_heating_on_zone1', 'is_heating_on_zone2',
       'target_temperature_zone1', 'target_temperature_zone2',
       'weather_temperature', 'weather_humidity', 'hot_water_control',
       'heating_control_zone1', 'heating_control_zone2', 'num_row_x',
       'serial_number', 'hub_serial_number', 'location_x',
       'internal_temperature', 'internal_humidity', 'num_row_y', 'location_y',
       'unit', 'value'],
      dtype='object')

In [15]:
merged_df = merged_df[['property_id',
                       'occurred',
                       'target_temperature_zone1',
                       'weather_temperature',
                       'weather_humidity',
                       'internal_temperature',
                       'internal_humidity','value']]

In [16]:
merged_df.head(3)

,property_id,occurred,target_temperature_zone1,weather_temperature,weather_humidity,internal_temperature,internal_humidity,value
0,20187,28/10/20 13:18,22.0,9.0,87.0,17.4,59.4,2
1,20187,28/10/20 13:29,22.0,9.0,87.0,18.0,57.7,2
2,20187,28/10/20 13:39,22.0,9.0,87.0,17.7,57.8,4


In [17]:
merged_df['occurred'] = pd.to_datetime(merged_df['occurred'],dayfirst=True)

/tmp/ipykernel_10033/601500091.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df['occurred'] = pd.to_datetime(merged_df['occurred'],dayfirst=True)


In [18]:
merged_df.head()

,property_id,occurred,target_temperature_zone1,weather_temperature,weather_humidity,internal_temperature,internal_humidity,value
0,20187,2020-10-28 13:18:00,22.0,9.0,87.0,17.4,59.4,2
1,20187,2020-10-28 13:29:00,22.0,9.0,87.0,18.0,57.7,2
2,20187,2020-10-28 13:39:00,22.0,9.0,87.0,17.7,57.8,4
3,20187,2020-10-28 13:49:00,22.0,9.0,87.0,17.7,58.1,5
4,20187,2020-10-28 13:59:00,25.0,9.0,87.0,17.8,58.5,8


In [19]:
merged_df = merged_df.drop_duplicates()

In [20]:
result = merged_df.copy()

In [21]:
property_list = merged_df['property_id'].unique()

In [22]:
all = []
for property in property_list:
    df = result[result['property_id'] == property]
    df.loc[:,'internal_temperature']  = df['internal_temperature'].fillna(method = 'ffill')
    df.loc[:,'internal_humidity'] = df['internal_humidity'].fillna(method = 'ffill')
    all.append(df)
final_results = pd.concat(all,ignore_index=True)

In [23]:
final_results.columns

Index(['property_id', 'occurred', 'target_temperature_zone1',
       'weather_temperature', 'weather_humidity', 'internal_temperature',
       'internal_humidity', 'value'],
      dtype='object')

In [24]:
daily_df = pickle.load(open('daily_df.pkl','rb'))

In [25]:
daily_df = daily_df[daily_df['property_id'].isin(property_list)]

In [26]:
daily_df.columns

Index(['property_id', 'hub_id', 'zone', 'id', 'type', 'occupied',
       'heating_system', 'heating_system_manufacturer',
       'heating_system_model_no', 'number_of_zones', 'occurred',
       'number_of_paired_sensors', 'lodgement_date',
       'current_energy_efficiency', 'property_type', 'built_form',
       'floor_description', 'windows_description', 'walls_description',
       'roof_description', 'number_habitable_rooms', 'number_heated_rooms',
       'total_floor_area', 'context_date', 'reported_hlr', 'reported_tth',
       'total_daily_sensor_readings', 'daily_avg_room_humidty',
       'daily_avg_room_temperature', 'daily_avg_weather_temperature',
       'daily_max_target_temperature', 'daily_total_delta_energy_meter_use',
       'daily_total_heating_use', 'reported_fuel_poverty_percentage',
       'daily_temp_below18c_total_sensor_readings', 'hub_type'],
      dtype='object')

In [27]:
daily_df_rdx = daily_df[['property_id','context_date',
                         'current_energy_efficiency',
                         'number_habitable_rooms',
                         'total_floor_area',
                         'reported_hlr','reported_tth','reported_fuel_poverty_percentage']]

In [28]:
final_results['context_date'] = final_results['occurred'].dt.date

In [29]:
final_results.head(2)

,property_id,occurred,target_temperature_zone1,weather_temperature,weather_humidity,internal_temperature,internal_humidity,value,context_date
0,20187,2020-10-28 13:18:00,22.0,9.0,87.0,17.4,59.4,2,2020-10-28
1,20187,2020-10-28 13:29:00,22.0,9.0,87.0,18.0,57.7,2,2020-10-28


In [30]:
merged_df.to_pickle('merged_telemetry.pkl')
daily_df_rdx.to_pickle('daily_df_rdx.pkl')